![polis](https://polisen.se/siteassets/bilder/polisen_twitter_image.png)
# Polisen.se - Get police events
version 2

With this API you can get events based on _when_, _where_ and _what_, for more details [read this](https://polisen.se/om-polisen/om-webbplatsen/oppna-data/api-over-polisens-handelser/).

- [API](https://polisen.se/kontakt/om-webbplatsen/oppna-data/api-over-polisens-handelser/)
  - [/events?locationname=Stockholm](https://polisen.se/api/events?locationname=Stockholm)
  - [/events?type=Brand](https://polisen.se/api/events?type=Brand)
  - [/events?DateTime=2020-05](https://polisen.se/api/events?DateTime=2020-05)
  - [/events?DateTime=2020-05&type=Stöld&locationname=Stockholm](https://polisen.se/api/events?DateTime=2020-05&type=Stöld&locationname=Stockholm)
 
GITHUB [this Notebook](https://github.com/salgo60/open-data-examples/blob/master/polisen-se.ipynb)


### Fundering: location.name  
Borde inte polisen vara tydligare med område som rapporteras om man skall kunna ha vettig statistik.... 



* Stockholms kommun samma som
  * WD [Q506250](https://www.wikidata.org/wiki/Q506250)
  * OSM [relation/398021](https://www.openstreetmap.org/relation/398021)
* Stockholms län etc...
  * WD [Q104231](https://www.wikidata.org/wiki/Q104231)
  * OSM [relation/54391](https://www.openstreetmap.org/relation/54391)
* Stockholms stad
  * WD [Q10680648](https://www.wikidata.org/wiki/Q10680648)

Större tydlighet av typ av brott och område skulle kunna göra att data kan jämföras mellan olika delar av världen och se hur olika brottsbekämpande åtgärder påverkar... 

**UPDATE** På Hack4Sweden hade jag en dialog med polisen och dom ser detta som ett problem och det finns vaga försök att ena med andra länder hur man rapporterar... 

**User Case** Genom att kunna klassificera områden på storlek befolkning, typ av område, social utsatthet etc... och matcha detta med insatser från samhället så borde det att tom i realtid kunna följa inrapporterade böcker från flera länder i världen i Realtid.... jmf tankarhur vetenskapliga priser skulle mha länkad data kunna användas för att utvärdera forskning i flera länder se [T216409](https://phabricator.wikimedia.org/T216409)  


In [1]:
from datetime import datetime
start_time  = datetime.now()
print("Last run: ", start_time)

Last run:  2020-10-20 20:57:39.109752


In [2]:
from IPython.display  import JSON
import requests
import json

baseUrl = "https://polisen.se/api"
endpoint = "/events?DateTime=2020-10&type=Stöld&locationname=Stockholm;Göteborg;Malmö;Uppsala"
endpoint = "/events?DateTime=2020-10&type=Stöld&locationname=Stockholm;Göteborg;Malmö;Uppsala"
url = baseUrl + endpoint
url

'https://polisen.se/api/events?DateTime=2020-10&type=Stöld&locationname=Stockholm;Göteborg;Malmö;Uppsala'

In [3]:
# Call the API
req = requests.get(url)

# Ensure we fail fast if the web request failed
req.raise_for_status()

data = json.loads(req.text)
JSON(data)

<IPython.core.display.JSON object>

In [4]:
import pandas as pd
from pandas.io.json import json_normalize

df = json_normalize(data)

<ipython-input-4-073446b6c521>:4: FutureWarning: pandas.io.json.json_normalize is deprecated, use pandas.json_normalize instead
  df = json_normalize(data)


In [5]:
# Show the first entries
df.head()

,id,datetime,name,summary,url,type,location.name,location.gps
0,230517,2020-10-18 14:41:00 +02:00,"18 oktober 14:41, Stöld, Malmö","Flera fall av stölder från cyklande personer, ...",https://polisen.se/aktuellt/handelser/2020/okt...,Stöld,Malmö,"55.604981,13.003822"
1,230481,2020-10-18 7:15:00 +02:00,"18 oktober 07:15, Stöld, Uppsala",Tre män grips misstänka för grov stöld.,https://polisen.se/aktuellt/handelser/2020/okt...,Stöld,Uppsala,"59.858564,17.638927"
2,230477,2020-10-18 1:53:00 +02:00,"18 oktober 01:53, Stöld, Stockholm",Två unga killar är gripna efter att ha stulit ...,https://polisen.se/aktuellt/handelser/2020/okt...,Stöld,Stockholm,"59.329324,18.068581"
3,230328,2020-10-16 16:06:00 +02:00,"16 oktober 16:06, Stöld, Stockholm",En person upptäcks när han stjäl en flaska spr...,https://polisen.se/aktuellt/handelser/2020/okt...,Stöld,Stockholm,"59.329324,18.068581"
4,230083,2020-10-15 10:54:00 +02:00,"15 oktober 10:54, Stöld, Uppsala",Oenighet kring ett auktionsköp leder bland ann...,https://polisen.se/aktuellt/handelser/2020/okt...,Stöld,Uppsala,"59.858564,17.638927"


In [6]:
# Create two new columns, one Latitude and one Longitude, using data in location.gps
df[['Latitude', 'Longitude']] = df['location.gps'].str.split(",", expand=True)
# Remove the old column
df.drop(labels='location.gps', axis=1)

,id,datetime,name,summary,url,type,location.name,Latitude,Longitude
0,230517,2020-10-18 14:41:00 +02:00,"18 oktober 14:41, Stöld, Malmö","Flera fall av stölder från cyklande personer, ...",https://polisen.se/aktuellt/handelser/2020/okt...,Stöld,Malmö,55.604981,13.003822
1,230481,2020-10-18 7:15:00 +02:00,"18 oktober 07:15, Stöld, Uppsala",Tre män grips misstänka för grov stöld.,https://polisen.se/aktuellt/handelser/2020/okt...,Stöld,Uppsala,59.858564,17.638927
2,230477,2020-10-18 1:53:00 +02:00,"18 oktober 01:53, Stöld, Stockholm",Två unga killar är gripna efter att ha stulit ...,https://polisen.se/aktuellt/handelser/2020/okt...,Stöld,Stockholm,59.329324,18.068581
3,230328,2020-10-16 16:06:00 +02:00,"16 oktober 16:06, Stöld, Stockholm",En person upptäcks när han stjäl en flaska spr...,https://polisen.se/aktuellt/handelser/2020/okt...,Stöld,Stockholm,59.329324,18.068581
4,230083,2020-10-15 10:54:00 +02:00,"15 oktober 10:54, Stöld, Uppsala",Oenighet kring ett auktionsköp leder bland ann...,https://polisen.se/aktuellt/handelser/2020/okt...,Stöld,Uppsala,59.858564,17.638927
5,229440,2020-10-12 10:57:00 +02:00,"12 oktober 10:57, Stöld, Stockholm",När en man som stulit i en butik på Kungsgatan...,https://polisen.se/aktuellt/handelser/2020/okt...,Stöld,Stockholm,59.329324,18.068581
6,228925,2020-10-07 18:10:00 +02:00,"07 oktober 18:10, Stöld, Stockholm",Cykelägaren återfann sin stulna cykel låst på ...,https://polisen.se/aktuellt/handelser/2020/okt...,Stöld,Stockholm,59.329324,18.068581
7,228765,2020-10-06 15:33:00 +02:00,"06 oktober 15:33, Stöld, Uppsala",Två män stjäl i en elektronikbutik.,https://polisen.se/aktuellt/handelser/2020/okt...,Stöld,Uppsala,59.858564,17.638927
8,228004,2020-10-01 2:30:00 +02:00,"01 oktober 02:30, Stöld, Malmö","Pågående cykelstöld, Oxie.",https://polisen.se/aktuellt/handelser/2020/okt...,Stöld,Malmö,55.604981,13.003822


In [7]:
# Show us the first few rows only 
df.head()

,id,datetime,name,summary,url,type,location.name,location.gps,Latitude,Longitude
0,230517,2020-10-18 14:41:00 +02:00,"18 oktober 14:41, Stöld, Malmö","Flera fall av stölder från cyklande personer, ...",https://polisen.se/aktuellt/handelser/2020/okt...,Stöld,Malmö,"55.604981,13.003822",55.604981,13.003822
1,230481,2020-10-18 7:15:00 +02:00,"18 oktober 07:15, Stöld, Uppsala",Tre män grips misstänka för grov stöld.,https://polisen.se/aktuellt/handelser/2020/okt...,Stöld,Uppsala,"59.858564,17.638927",59.858564,17.638927
2,230477,2020-10-18 1:53:00 +02:00,"18 oktober 01:53, Stöld, Stockholm",Två unga killar är gripna efter att ha stulit ...,https://polisen.se/aktuellt/handelser/2020/okt...,Stöld,Stockholm,"59.329324,18.068581",59.329324,18.068581
3,230328,2020-10-16 16:06:00 +02:00,"16 oktober 16:06, Stöld, Stockholm",En person upptäcks när han stjäl en flaska spr...,https://polisen.se/aktuellt/handelser/2020/okt...,Stöld,Stockholm,"59.329324,18.068581",59.329324,18.068581
4,230083,2020-10-15 10:54:00 +02:00,"15 oktober 10:54, Stöld, Uppsala",Oenighet kring ett auktionsköp leder bland ann...,https://polisen.se/aktuellt/handelser/2020/okt...,Stöld,Uppsala,"59.858564,17.638927",59.858564,17.638927


In [8]:
df["type"].value_counts()

Stöld    9
Name: type, dtype: int64

In [9]:
df.groupby(["location.name", "type"]).size()

location.name  type 
Malmö          Stöld    2
Stockholm      Stöld    4
Uppsala        Stöld    3
dtype: int64

In [10]:
df.groupby(["type", "location.name"]).size()

type   location.name
Stöld  Malmö            2
       Stockholm        4
       Uppsala          3
dtype: int64

# Policestation API
see  about [api-over-polisstationer](https://polisen.se/om-polisen/om-webbplatsen/oppna-data/api-over-polisstationer/)

In [11]:
baseUrl = "https://polisen.se/api"
endpoint = "/policestations"
urlPolisStation= baseUrl + endpoint
urlPolisStation


'https://polisen.se/api/policestations'

In [12]:
# Call the API
reqPolisStation = requests.get(urlPolisStation)

# Ensure we fail fast if the web request failed
reqPolisStation.raise_for_status()

dataPolisStation = json.loads(reqPolisStation.text)
JSON(dataPolisStation)

<IPython.core.display.JSON object>

In [13]:
import pandas as pd
from pandas.io.json import json_normalize
dfPolisStations = json_normalize(dataPolisStation)

<ipython-input-13-89b82680ec58>:3: FutureWarning: pandas.io.json.json_normalize is deprecated, use pandas.json_normalize instead
  dfPolisStations = json_normalize(dataPolisStation)


In [14]:
dfPolisStations.head()

,id,name,Url,services,location.name,location.gps
0,1233,Alingsås,https://polisen.se/kontakt/polisstationer/vast...,"[{'name': 'Anmälan'}, {'name': 'Cyklar'}, {'na...","N Strömgatan 8, Alingsås","57.930105,12.529608"
1,811,Alvesta,https://polisen.se/kontakt/polisstationer/kron...,"[{'name': 'Anmälan'}, {'name': 'Hittegods'}]","Värendsgatan 14, Alvesta","56.900514,14.555233"
2,7519,Aneby,https://polisen.se/kontakt/polisstationer/jonk...,"[{'name': 'Anmälan'}, {'name': 'Hittegods'}]","Stigbergsvägen 11, Aneby","57.836076,14.807909"
3,1216,Arboga,https://polisen.se/kontakt/polisstationer/vast...,"[{'name': 'Anmälan'}, {'name': 'Hittegods'}, {...","Järntorget 8, Arboga","59.39417,15.840519"
4,832,Arjeplog,https://polisen.se/kontakt/polisstationer/norr...,"[{'name': 'Anmälan'}, {'name': 'Cyklar'}, {'na...","Tingsbacka, Storgatan 20 , Arjeplog","66.052216,17.893128"


In [15]:
dfPolisStations[['Latitude', 'Longitude']] = dfPolisStations['location.gps'].str.split(",", expand=True)

In [16]:
from geojson import Point 

In [17]:
# Convert a pandas dataframe to geojson
geojsonPolice = {'type':'FeatureCollection', 'features':[]}
for i, row in enumerate(dfPolisStations.values):
    features = []
    #print(dfPolisStations.Latitude[i],dfPolisStations.Longitude[i],dfPolisStations.name[i])
    feature = {'type':'Feature', 'properties':{},'geometry':{'type':'Point','coordinates':[]}}   
    # fill in the coordinates
    feature['geometry']['coordinates'] = [dfPolisStations.Longitude[i],dfPolisStations.Latitude[i]]
    feature['properties']['name'] = dfPolisStations.name[i]
        
    # add this feature to the list of features inside our dict
    geojsonPolice['features'].append(feature)
#print(json.dumps(geojsonPolice, indent=2))

In [18]:
from ipyleaflet import * 
m = Map(center=(59.3,18.3), zoom=6)
geo_json = GeoJSON(data=geojsonPolice, 
                   style = {'fillColor': 'blue'})
m.add_layer(geo_json)
m

Map(center=[59.3, 18.3], controls=(ZoomControl(options=['position', 'zoom_in_text', 'zoom_in_title', 'zoom_out…

![alt text](OutputPolice.png "Output")

In [19]:
m = Map(basemap=basemaps.OpenTopoMap,center=(59.3,18.3), zoom=6)
m.add_layer(geo_json) 
m

Map(center=[59.3, 18.3], controls=(ZoomControl(options=['position', 'zoom_in_text', 'zoom_in_title', 'zoom_out…

In [20]:
m.add_control(FullScreenControl()) 


Add measurement

In [21]:
measure = MeasureControl(
    position='bottomleft',
    active_color = 'orange',
    primary_length_unit = 'kilometers'
) 
m.add_control(measure)
m


Map(center=[59.3, 18.3], controls=(ZoomControl(options=['position', 'zoom_in_text', 'zoom_in_title', 'zoom_out…

In [22]:
marker = Marker(icon=AwesomeIcon(name="check", marker_color='green', icon_color='darkgreen'))

m.add_control(SearchControl(
  position="topleft",
  url='https://nominatim.openstreetmap.org/search?format=json&q={s}',
  zoom=5,
  marker=marker
))

m

Map(center=[59.3, 18.3], controls=(ZoomControl(options=['position', 'zoom_in_text', 'zoom_in_title', 'zoom_out…